In [ ]:
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


In [ ]:
# 🤖 Welcome Message
display(Markdown("## 👋 Hey there! I’m *CryptoBuddy* – your friendly AI crypto sidekick!"))
display(Markdown("### You can:"))
display(Markdown("- Click buttons for common questions\n- Choose a coin to view its stats\n- Type your own question"))


In [ ]:
# 💰 Crypto Database
crypto_db = {
    "Bitcoin": {
        "price_trend": "rising",
        "market_cap": "high",
        "energy_use": "high",
        "sustainability_score": 3
    },
    "Ethereum": {
        "price_trend": "stable",
        "market_cap": "high",
        "energy_use": "medium",
        "sustainability_score": 6
    },
    "Cardano": {
        "price_trend": "rising",
        "market_cap": "medium",
        "energy_use": "low",
        "sustainability_score": 8
    },
    "Solana": {
        "price_trend": "rising",
        "market_cap": "medium",
        "energy_use": "low",
        "sustainability_score": 7
    },
    "Dogecoin": {
        "price_trend": "falling",
        "market_cap": "low",
        "energy_use": "medium",
        "sustainability_score": 4
    }
}

In [ ]:
# Bot Logic
def get_response(user_query):
    query = user_query.lower().strip()

    if not query:
        return None

    if query in ['exit', 'quit']:
        return "👋 Bye! Remember, crypto is risky—do your own research! 🚨"

    if "sustainable" in query:
        best = max(crypto_db, key=lambda x: crypto_db[x]["sustainability_score"])
        return f"🌱 *{best}* is the most eco-friendly choice!"

    elif "trending" in query or "rising" in query:
        rising = [name for name, data in crypto_db.items() if data["price_trend"] == "rising"]
        return f"📈 Trending up: *{', '.join(rising)}*."

    elif "long-term" in query or "growth" in query:
        good_choices = [name for name, data in crypto_db.items() if data["price_trend"] == "rising" and data["sustainability_score"] >= 7]
        return f"🚀 Best for long-term: *{', '.join(good_choices)}*." if good_choices else " No strong long-term growth coins right now."

    elif "profitable" in query or "best investment" in query:
        candidates = [name for name, data in crypto_db.items() if data["price_trend"] == "rising" and data["market_cap"] == "high"]
        return f"💰 High potential: *{', '.join(candidates)}*." if candidates else "📉 No profitable picks today."

    return " I didn’t get that. Try: sustainable, trending, long-term, or profitable."

In [ ]:
# 📊 Chart Helpers
def convert_to_score(attribute, value):
    scale = {
        'market_cap': {'low': 3, 'medium': 6, 'high': 9},
        'price_trend': {'falling': 3, 'stable': 6, 'rising': 9},
        'energy_use': {'high': 3, 'medium': 6, 'low': 9}
    }
    return scale.get(attribute, {}).get(value, 0)

def plot_comparison(metric_type):
    output_area.clear_output()
    
    with output_area:
        coins = list(crypto_db.keys())

        if metric_type == 'sustainability':
            scores = [crypto_db[coin]['sustainability_score'] for coin in coins]
            title = "🌿 Sustainability Score"
            ylabel = "Score (0-10)"
            colors = ['green' if s >= 7 else 'orange' if s >= 5 else 'red' for s in scores]

        elif metric_type == 'profitability':
            scores = [convert_to_score('market_cap', crypto_db[coin]['market_cap']) +
                      convert_to_score('price_trend', crypto_db[coin]['price_trend']) for coin in coins]
            title = "💰 Estimated Profitability"
            ylabel = "Score (max 18)"
            colors = ['gold' if s >= 15 else 'orange' if s >= 10 else 'gray' for s in scores]

        elif metric_type == 'long_term':
            scores = [convert_to_score('price_trend', crypto_db[coin]['price_trend']) +
                      crypto_db[coin]['sustainability_score'] for coin in coins]
            title = "📈 Long-Term Growth Potential"
            ylabel = "Score (max 19)"
            colors = ['blue' if s >= 15 else 'purple' if s >= 10 else 'gray' for s in scores]

        elif metric_type == 'trending':
            scores = [convert_to_score('price_trend', crypto_db[coin]['price_trend']) for coin in coins]
            title = "🔥 Price Trend Score"
            ylabel = "Score (0-10)"
            colors = ['red' if s >= 7 else 'orange' if s >= 5 else 'gray' for s in scores]

        else:
            display(Markdown("❓ I can't create a chart for that input."))
            return

        plt.figure(figsize=(8, 4))
        bars = plt.bar(coins, scores, color=colors)
        plt.ylim(0, max(scores) + 2)
        plt.title(title)
        plt.ylabel(ylabel)
        plt.grid(axis='y', linestyle='--', alpha=0.4)

        for bar, score in zip(bars, scores):
            yval = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2.0, yval + 0.2, f'{score}', ha='center', va='bottom')

        plt.tight_layout()
        plt.show()
        display(Markdown(f"**CryptoBuddy:** Here's how the coins compare on **{metric_type.replace('_', ' ')}**."))
        

In [ ]:
# 🔘 Common Questions Buttons
button_questions = {
"🌱 Most Sustainable Coin": "sustainability",
"📈 Trending Coins": "trending",
"🚀 Best for Long-Term": "long_term",
"💰 Most Profitable Now": "profitability"
}

button_widgets = []
for label, query_key in button_questions.items():
btn = widgets.Button(description=label, layout=widgets.Layout(width='auto'))
def on_click_handler(b, key=query_key):
output_area.clear_output()
response = get_response(key)
with output_area:
display(Markdown(f"**CryptoBuddy:** {response}"))
plot_comparison(key)
btn.on_click(on_click_handler)
button_widgets.append(btn)

buttons_box = widgets.HBox(button_widgets)

# 📦 Dropdown for Coin Details
coin_dropdown = widgets.Dropdown(
options=list(crypto_db.keys()),
description='🔍 Coin:',
layout=widgets.Layout(width='50%')
)

def show_coin_info(change):
coin = change['new']
if coin:
data = crypto_db[coin]
with output_area:
output_area.clear_output()
display(Markdown(f"""
**{coin} Info:**
- 📈 Price Trend: `{data['price_trend']}`
- 💰 Market Cap: `{data['market_cap']}`
- ⚡ Energy Use: `{data['energy_use']}`
- 🌍 Sustainability Score: `{data['sustainability_score']}/10`
"""))

coin_dropdown.observe(show_coin_info, names='value')
